Get mean and standard deviation for transforms i.e. PyTorch's "transforms.Normalize". <br>
This normalizes the tensor image with mean and standard deviation. <br>

In [1]:
# Import libraries
import os
import pickle

import numpy as np
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms
from tqdm import tqdm




In [2]:
# Specify classes
classes = (
    "Preparation_trocar_insertion",
    "Calot_triangle_dissection",
    "Clipping_and_cutting",
    "Gallbladder_dissection",
    "Gallbladder_packaging",
    "Cleaning_and_coagulation",
    "Gallbladder_extraction",
)


class Cholec80Dataset(Dataset):
    '''
    Dataset Definition 
    '''
    def __init__(self, df, transform=None):
        self.image_path = df['image']
        self.phase_annotations = df['phase']
        self.tool_annotations = df[[
        'tool_Grasper',
        'tool_Bipolar',
        'tool_Hook',
        'tool_Scissors',
        'tool_Clipper',
        'tool_Irrigator',
        'tool_SpecimenBag']]
        self.transform = transform

    def __len__(self):
        return len(self.phase_annotations)


    def __getitem__(self, index):
        img_path = self.image_path[index]
        img = Image.open(img_path).convert("RGB")
        phase_label = self.phase_annotations[index]

        tool_label = self.tool_annotations.iloc[index].values
        tool_label = tool_label.tolist()
        tool_label = torch.FloatTensor(tool_label)
        if self.transform is not None:
            img = self.transform(img)

        return (img, img_path, phase_label, tool_label)

In [4]:
def _get_transform():
    """All pre-trained models expect input images normalized in the same way,
       i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H 
       and W are expected to be at least 224."""
    return transforms.Compose(
        [ transforms.CenterCrop(224), transforms.ToTensor(),]
    )

# Get mean and standard deviation for overall test set

In [5]:
#Import train, valdiation and test sets
train_df = pd.read_parquet('data/train_df.parquet')
val_df = pd.read_parquet('data/val_df.parquet')
test_df = pd.read_parquet('data/test_df.parquet')



train_df = train_df[train_df['frame'] % 25 == 0].reset_index(drop=True)
val_df = val_df[val_df['frame'] % 25 == 0].reset_index(drop=True)
test_df = test_df[test_df['frame'] % 25 == 0].reset_index(drop=True)



train_dataset = Cholec80Dataset(train_df, _get_transform())
val_dataset = Cholec80Dataset(val_df, _get_transform())
test_dataset = Cholec80Dataset(test_df, _get_transform())

In [11]:
def get_mean_std(dataset):
    mean = []
    std = []

    for i in tqdm(range(len(dataset))):
        img_tensor = dataset[i][0]
        image = img_tensor.numpy()

        # Batch mean
        batch_mean = np.mean(image, axis=(1, 2))
        batch_std = np.std(image, axis=(1, 2))

        mean.append(batch_mean)
        std.append(batch_std)

    # Reshape it
    mean = np.array(mean).mean(axis=0)
    std = np.array(std).mean(axis=0)

    return mean, std 

### Train mean and std

In [4]:
train_mean, train_std = get_mean_std(train_dataset)
print(train_mean,train_std)

### Validation mean and std

In [14]:
val_mean, val_std = get_mean_std(val_dataset)
print(val_mean,val_std)

100%|██████████| 23770/23770 [01:22<00:00, 288.59it/s]


[0.4381856  0.2710791  0.27274346] [0.21493198 0.20029972 0.19810152]


### Test mean and std

In [15]:
test_mean, test_std = get_mean_std(test_dataset)
print(test_mean,test_std)

100%|██████████| 23379/23379 [01:28<00:00, 263.47it/s]


[0.3904725  0.2648226  0.25951037] [0.17919387 0.16188985 0.15494719]
